In [ ]:
import firedrake
from firedrake import max_value, sqrt, exp, Constant

mesh = firedrake.UnitDiskMesh(5)
R = 250e3
mesh.coordinates.dat.data[:] *= R

cg1 = firedrake.FiniteElement("CG", "triangle", 1)
Q = firedrake.FunctionSpace(mesh, cg1)

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)
r = sqrt(x**2 + y**2)

# Plateau elevation
b_base = Constant(400)

# Max elevation
b_max = Constant(1400)

# Radius of the plateau interior
ro = 125e3

# Radius of the ridge
Ro = Constant(200e3)

def tanh(z):
    return (exp(z) - exp(-z)) / (exp(z) + exp(-z))

def θ(z):
    return (tanh(z) + 1) / 2

def sech(z):
    return 2 / (exp(z) + exp(-z))

a = Constant(50e3)
ξ = (sqrt(x**2 + y**2) - ro) / a
ζ = (r - Ro) / Ro

ρ1 = Constant(1 / 4)
μ1 = 1 - ρ1 * θ(3 * (x - ro / 4) / a) * sech(2 * y / a)

ρ2 = Constant(3 / 8)
μ2 = 1 - ρ2 * θ(3 * (y - ro / 4) / a) * sech(2 * x / a)

ρ3 = Constant(1 / 2)
μ3 = 1 - ρ3 * θ(3 * (-x + ro / 4) / a) * sech(2 * y / a)

ρ4 = Constant(5 / 8)
μ4 = 1 - ρ4 * θ(3 * (-y + ro / 4) / a) * sech(2 * x / a)

μ = μ1 * μ2 * μ3 * μ4

S = 480 / (1 - Ro / R)

b_expr_plateau = b_base * (1 - θ(3 * ξ))
b_expr_valleys = (b_max - b_base) * sech(3 * ξ) * μ - θ(5 * ζ) * S * ζ
b_expr = max_value(0, b_expr_plateau + b_expr_valleys)
b = firedrake.interpolate(b_expr, Q)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
firedrake.trisurf(b, axes=ax);

In [ ]:
max_radius = Constant(195e3)
dome_height = Constant(2.4e3)
dome = dome_height * max_value(1 - (x**2 + y**2) / max_radius**2, 0)
s = firedrake.interpolate(dome, Q)

In [ ]:
h = firedrake.interpolate(s - b, Q)
h_0 = h.copy(deepcopy=True)

In [ ]:
b3 = firedrake.FiniteElement("B", "triangle", 3)
V = firedrake.VectorFunctionSpace(mesh, cg1 + b3)

In [ ]:
import icepack
from icepack.constants import gravity as g, ice_density as ρ_I
from firedrake import inner, grad, dx

T = Constant(260.0)
A = icepack.rate_factor(T)
n = Constant(3.0)

u = firedrake.Function(V)
v = firedrake.TestFunction(V)

mass = inner(u, v) * dx
P = ρ_I * g * h
S_n = inner(grad(s), grad(s))**((n - 1) / 2)
gravity = 2 * A * P ** n / (n + 2) * h * S_n * inner(grad(s), v) * dx
F = mass + gravity

In [ ]:
solver_params = {"snes_type": "ksponly", "ksp_type": "gmres"}
fc_params = {"quadrature_degree": 6}
params = {"solver_parameters": solver_params, "form_compiler_parameters": fc_params}
firedrake.solve(F == 0, u, **params)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(firedrake.project(sqrt(inner(u, u)), Q), axes=ax)
fig.colorbar(colors);

In [ ]:
Z = Q * V * V
z = firedrake.Function(Z)
z_n = firedrake.Function(Z)

z.sub(0).interpolate(s - b)
z.sub(1).assign(u)
z.sub(2).project(h * u)

z_n.assign(z);

In [ ]:
h, u, q = firedrake.split(z)
h_n = firedrake.split(z_n)[0]
η, v, w = firedrake.TestFunctions(Z)

dt = firedrake.Constant(10.0)

s = b + h
F_h = ((h - h_n) * η - dt * inner(q, grad(η))) * dx
P = ρ_I * g * h
# TODO: look at the codegen when we break up this computation differently
S_n = inner(grad(s), grad(s))**((n - 1) / 2)
F_u = inner(u + 2 * A * P**n / (n + 2) * h * S_n * grad(s), v) * dx
F_q = inner(q - h * u, w) * dx

F = F_h + F_u + F_q

problem = firedrake.NonlinearVariationalProblem(F, z)
params = {
    "solver_parameters": {
        "snes_type": "ksponly",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    }
}
solver = firedrake.NonlinearVariationalSolver(problem, **params)

In [ ]:
import tqdm

hs = [z.sub(0).copy(deepcopy=True)]

final_time = 10e3
num_steps = int(final_time / float(dt))
for step in tqdm.trange(num_steps):
    solver.solve()
    z.sub(0).interpolate(max_value(0, z.sub(0)))
    z_n.assign(z)
    hs.append(z.sub(0).copy(deepcopy=True))

In [ ]:
h, u, q = z.split()
h.dat.data_ro.min(), h.dat.data_ro.max()

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(h, axes=ax)
fig.colorbar(colors);

In [ ]:
initial_volume = firedrake.assemble(h_0 * dx) / 1e9
volume = firedrake.assemble(h * dx) / 1e9
print(f"Initial volume:  {initial_volume:g}km³")
print(f"Final volume:    {volume:g}km³")
print(f"Relative change: {(volume - initial_volume) / initial_volume}")

In [ ]:
from firedrake.plot import FunctionPlotter
fn_plotter = FunctionPlotter(mesh, num_sample_points=1)

In [ ]:
%%capture
fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
colors = firedrake.tripcolor(
    hs[0], num_sample_points=1, vmin=0, vmax=2e3, shading="gouraud", axes=axes
)

from matplotlib.animation import FuncAnimation
def animate(h):
    colors.set_array(fn_plotter(h))

interval = 1e3 / 25
animation = FuncAnimation(fig, animate, frames=hs, interval=interval)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())